In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
import json
import scipy.optimize
from predict import predict_sigma_vcosmo

In [153]:
# optimized parameters
q0 = 79.53 # [A^2]
r0 = 66.69 # [A^3]
z_coordination = 10
c_hb = 14873.69 # kcal A^4 / mol/e^2
R = 8.3144598/4184 # 0.001987 # but really: 8.3144598/4184
sigma_hb = 0.0104
EPS = 96.55 # (LIN AND SANDLER USE A CONSTANT FPOL WHICH YIELDS EPS=3.68)3.667
AEFFPRIME = 3.74
EO = 2.395e-4
FPOL = (EPS-1.0)/(EPS+0.5)
ALPHA = (0.3*AEFFPRIME**(1.5))/(EO)
alpha_prime = FPOL*ALPHA
print(alpha_prime, R)



# original parameters
# q0 = 79.53 # [A^2]
# r0 = 66.69 # [A^3]
# z_coordination = 10
# c_hb = 58250.49 # kcal A^4 / mol/e^2
# R = 8.3144598/4184 # 0.001987 # but really: 8.3144598/4184
# sigma_hb = 0.01008
# EPS = 3.667 # (LIN AND SANDLER USE A CONSTANT FPOL WHICH YIELDS EPS=3.68)3.667
# AEFFPRIME = 7.5
# EO = 2.395e-4
# FPOL = (EPS-1.0)/(EPS+0.5)
# ALPHA = (0.3*AEFFPRIME**(1.5))/(EO)
# alpha_prime = FPOL*ALPHA
# print(alpha_prime, R)

8919.865214682537 0.001987203585086042


In [154]:
sigma_tabulated = np.linspace(-0.025, 0.025, 51)
sigma_m = np.tile(sigma_tabulated,(len(sigma_tabulated),1))
sigma_n = np.tile(np.array(sigma_tabulated,ndmin=2).T,(1,len(sigma_tabulated)))
sigma_acc = np.tril(sigma_n) + np.triu(sigma_m,1)
sigma_don = np.tril(sigma_m) + np.triu(sigma_n,1)
DELTAW = (alpha_prime/2)*(sigma_m + sigma_n)**2+ c_hb*np.maximum(0, sigma_acc - sigma_hb)*np.minimum(0, sigma_don+sigma_hb)

In [ ]:

def get_sigma_profile(smiles):

    pred_sigma_df, pred_vcosmo = predict_sigma_vcosmo(smiles,draw=False)  
    V_COSMO = float(pred_vcosmo)

    # 获取sigma_1到sigma_51的列数据
    sigma_columns = [f'sigma_{i}' for i in range(1, 52)]  # sigma_1 到 sigma_51
    sigma_values = pred_sigma_df[sigma_columns].values.flatten()
    
    # 将p(sigma)*A [A^2]数据转为DataFrame格式
    dd = pd.DataFrame(sigma_values, columns=['p(sigma)*A [A^2]'])

    # 计算A（p(sigma)*A [A^2]的总和）
    dd['A'] = dd['p(sigma)*A [A^2]'].sum()

    # 计算p(sigma)
    dd['p(sigma)'] = dd['p(sigma)*A [A^2]'] / dd['A']

    # 生成sigma [e/A^2]列，从-0.025到0.025，间隔为0.001
    sigma_range = np.arange(-0.025, 0.026, 0.001)  # 生成从-0.025到0.025的数组，包含0.025
    dd['sigma [e/A^2]'] = sigma_range

    return dd, V_COSMO


In [156]:

def get_Gamma(T, psigma):
    """计算 Gamma（表面活性系数）"""
    Gamma = np.ones_like(psigma)
    AA = np.exp(-DELTAW / (R * T)) * psigma
    for _ in range(50):
        Gammanew = 1 / np.sum(AA * Gamma, axis=1)
        difference = np.abs((Gamma - Gammanew) / (Gamma + 1e-10))
        Gamma = (Gammanew + Gamma) / 2
        if np.max(difference) < 1e-8:
            break
    return Gamma

def get_lngamma_resid(T, i, psigma_mix, prof, lnGamma_mix=None):
    """计算残余部分的 ln(γ_i)"""
    if lnGamma_mix is None:
        lnGamma_mix = np.log(get_Gamma(T, psigma_mix))

    psigma = prof['p(sigma)'].values
    A_i = prof['A'].iloc[0]
    lnGammai = np.log(get_Gamma(T, psigma))
    lngammai = A_i / AEFFPRIME * np.sum(psigma * (lnGamma_mix - lnGammai))
    return lngammai

def get_lngamma_comb(T, x, i, profs, V_COSMO_A3):
    """计算组合部分的 ln(γ_i)"""
    A = np.array([prof['p(sigma)*A [A^2]'].sum() for prof in profs])
    q = A / q0
    r = V_COSMO_A3 / r0
    theta_i = x[i] * q[i] / np.dot(x, q)
    phi_i = x[i] * r[i] / np.dot(x, r)
    l = z_coordination / 2 * (r - q) - (r - 1)
    return (np.log(phi_i / x[i]) + z_coordination / 2 * q[i] * np.log(theta_i / phi_i)
            + l[i] - phi_i / x[i] * np.dot(x, l))

def get_lngamma(T, x, i, psigma_mix, profs, V_COSMO_A3, lnGamma_mix=None):
    """计算总的 ln(γ_i)"""
    return (get_lngamma_resid(T, i, psigma_mix, profs[i], lnGamma_mix)
            + get_lngamma_comb(T, x, i, profs, V_COSMO_A3))

In [157]:
def predict_AC(solute_smiles, solvent_smiles, T, x_solute=1e-8):
    """预测无限稀释活度系数 (IDAC)"""
    smiles_list = [solute_smiles, solvent_smiles]
    x = np.array([x_solute, 1 - x_solute])

    # 获取 sigma 分布和 Vcosmo
    profs, V_COSMO_A3 = zip(*[get_sigma_profile(smiles) for smiles in smiles_list])
    profs = list(profs)
    V_COSMO_A3 = np.array(V_COSMO_A3)

    # 计算混合物的 psigma_mix
    As = [prof['p(sigma)*A [A^2]'].sum() for prof in profs]
    psigma_mix = sum(x[i] * profs[i]['p(sigma)*A [A^2]'] for i in range(2))
    psigma_mix /= sum(x[i] * As[i] for i in range(2))

    # 计算 lnGamma_mix（只计算一次）
    lnGamma_mix = np.log(get_Gamma(T, psigma_mix.values))

    # 计算溶质的 ln(γ)
    lngamma1 = get_lngamma(T, x, 0, psigma_mix, profs, V_COSMO_A3, lnGamma_mix)
    lngamma2 = get_lngamma(T, x, 1, psigma_mix, profs, V_COSMO_A3, lnGamma_mix)
    return lngamma1,lngamma2


In [158]:
predict_IDAC = predict_AC(solute_smiles='CC(Cl)Cl', solvent_smiles='CCCCCCCCCCCCCC(=O)N(C)C', T = 326.35)

In [159]:
predict_IDAC

(-0.7191742505596098, -3.889348638674007e-08)